In [ ]:
import numpy as np
import time

start_time = time.time()


def act(x):
    return 0 if x < 0.5 else 1


def go(my_list): # перепишем ф-цию, чтобы она могла принимать на вход лист
    x = np.array(my_list)
    w11 = [0.5, 0.5, 0, 0,-0.5, 0, 0]
    w12 = [0, 0, 0, -0.2, 0, 0.7, -0.3]
    w13 = [0, 0.3, 0.4, 0.3, -0.5, 0, 0]
    weight1 = np.array([w11, w12, w13]) # матрица 3x7
    weight2 = np.array([0.5, -0.4, 0.5]) # вектор 1х3

    sum_hidden = np.dot(weight1, x) # вычисляем сумму на входах нейронов
    # скрытого слоя
    #print("Значения сумм на нейронах скрытого слоя: "+str(sum_hidden))
    out_hidden = np.array([act(x) for x in sum_hidden])
    #print("Значения на выходах нейронов скрытого слоя: "+str(out_hidden))
    sum_end = np.dot(weight2, out_hidden)
    y = act(sum_end)
    #print("Выходное значение НС: "+str(y))
    return y


def generate_data():
    my_data = []
    n = 0
    while '{0:07b}'.format(n) != '{0:07b}'.format(128):
        my_data.append(list('{0:07b}'.format(n)))
        n += 1
    return my_data

data = generate_data()
output = []
for i in data:
    local = []
    for j in i: #создадим цикл для формирования локального листа с интеджерами 
        local.append(int(j)) # так как наша функция возвращает лист листов строк
    local.append(go(local)) # а для использования данных в ф-ции нс необходимы интеджеры
    output.append(tuple(local)) #плюс сразу используем эту переменную для формирования листа кортежей

    
def f(x):
    return 2/(1 + np.exp(-x)) - 1

def df(x):
    return 0.5*(1 + x)*(1 - x)

W1 = np.array([[-0.2, 0.3, -0.4, 0, 0, 0, 0], [0.1, 0, 0, -0.3, 0, 0, -0.4], [0, 0, 0, 0, 0, -0.4, 0.1]])
W2 = np.array([0.2, 0.3, 0.5])

def go_forward(inp):
    sum = np.dot(W1, inp)
    out = np.array([f(x) for x in sum])

    sum = np.dot(W2, out)
    y = f(sum)
    return (y, out)

def train(epoch):
    global W2, W1
    lmd = 0.001          # шаг обучения
    N = 10000000           # число итераций при обучении
    count = len(epoch)
    for k in range(N):
        x = epoch[np.random.randint(0, count)]  # случайных выбор входного сигнала из обучающей выборки
        y, out = go_forward(x[0:7])             # прямой проход по НС и вычисление выходных значений нейронов
        e = y - x[-1]                           # ошибка
        delta = e*df(y)                         # локальный градиент
        W2[0] = W2[0] - lmd * delta * out[0]    # корректировка веса первой связи
        W2[1] = W2[1] - lmd * delta * out[1]    # корректировка веса второй связи
        W2[2] = W2[2] - lmd * delta * out[2]    # корректировка веса третьей связи
        
        delta2 = W2*delta*df(out)               # вектор из 3-х величин локальных градиентов

        # корректировка связей первого слоя
        W1[0, :] = W1[0, :] - np.array(x[0:7]) * delta2[0] * lmd
        W1[1, :] = W1[1, :] - np.array(x[0:7]) * delta2[1] * lmd
        W1[2, :] = W1[2, :] - np.array(x[0:7]) * delta2[2] * lmd
        
# обучающая выборка (она же полная выборка)
epoch = output

train(epoch)        # запуск обучения сети

# проверка полученных результатов
final = []
for x in epoch:
    y, out = go_forward(x[0:7])
    if round(y) == x[-1] or round(y) == -1:
        final.append(1)
    else:
        final.append(0)
    print(f"Выходное значение НС: {y} => {x[-1]}", round(y) == x[-1] or round(y) == -1)
print()
print("Веса: ", W1, W2)
print("Процент ошибки:", final.count(0)/len(final) * 100)
print("--- %s seconds ---" % (time.time() - start_time))